In [ ]:
if __name__ == '__main__':
	main()

Bob is assigned to H1
Sally is assigned to H3
Billy is assigned to H2
Mary is assigned to H1
Lilly is assigned to H1


In [ ]:
# Generate Cost Matrix from User Input

import numpy as np
import pandas as pd

def get_integer_input(prompt):
    while True:
        try:
            return int(input(prompt))
        except ValueError:
            print("Invalid input. Please enter a valid integer.")

def get_list_input(prompt, delimiter=','):
    user_input = input(prompt)
    items = [item.strip() for item in user_input.split(delimiter)]
    return items

def get_hospitals_with_capacities():
    hospital_data = get_list_input("Enter hospital names and capacities (name1,capacity1,name2,capacity2,...):")

    hospitals = []
    capacities = []

    # Extract hospital names and capacities from the input list
    for i in range(0, len(hospital_data), 2):
        hospital_name = hospital_data[i]
        capacity = int(hospital_data[i + 1])
        hospitals.append(hospital_name)
        capacities.append(capacity)

    return hospitals, capacities

# Get the number of residents
num_residents = get_integer_input("Enter the number of residents to be assigned: ")

# Get hospitals and their capacities as separate lists
hospital_names, hospital_capacities = get_hospitals_with_capacities()

residents = []

# Get rankings from each resident
rankings = []

for i in range(num_residents):
    resident_name = input(f"Enter the name of resident {i + 1}: ")
    residents.append(resident_name)

    resident_rankings = get_list_input(f"Enter hospital rankings for {resident_name} (comma-separated):")
    resident_rankings = [int(rank) for rank in resident_rankings]
    rankings.append(resident_rankings)

# Create a cost matrix from the 'rankings' list
cost_matrix = np.array(rankings)

# Convert the NumPy matrix to a Pandas dataframe by row
df = pd.DataFrame(cost_matrix, columns= hospital_names)
df.index = residents

# display(df)

Enter the number of residents to be assigned: 5
Enter hospital names and capacities (name1,capacity1,name2,capacity2,...):H1, 3, H2, 2, H3, 4
Enter the name of resident 1: Bob
Enter hospital rankings for Bob (comma-separated):1,1,1
Enter the name of resident 2: Sally
Enter hospital rankings for Sally (comma-separated):2,2,1
Enter the name of resident 3: Billy
Enter hospital rankings for Billy (comma-separated):3,1,2
Enter the name of resident 4: Mary
Enter hospital rankings for Mary (comma-separated):3,3,3
Enter the name of resident 5: Lilly
Enter hospital rankings for Lilly (comma-separated):1,1,2


,H1,H2,H3
Bob,1,1,1
Sally,2,2,1
Billy,3,1,2
Mary,3,3,3
Lilly,1,1,2


In [ ]:
#Take in dataframe of each doctor's ranking of hospitals and a list of capacities of the hospitals and output an adjusted square ranks numpy matrix
#and hospital names
def matrix_with_capacities(ranks,capacities):
  #duplicate columns by the hospital's capacity
	for columnName,capacity in zip(ranks,capacities):
		for i in range(capacity-1):
			ranks[columnName + '.' + str(i+2)] = ranks.loc[:, columnName]
	ranks = ranks.reindex(sorted(ranks.columns), axis=1) #sort columns by hospital name
	col_names = ranks.columns
	num_doctors = ranks.shape[0]
	num_hospitals = ranks.shape[1]
  #add dummy rows to make matrix square
	for i in range(num_hospitals - num_doctors):
		ranks.loc[len(ranks)] = [ranks.values.max()+1]*num_hospitals
	num_doctors = ranks.shape[0]
	num_hospitals = ranks.shape[1]
	cost_matrix = ranks.to_numpy()
	return cost_matrix, col_names

In [ ]:
def min_zero_row(zero_mat, mark_zero):

	'''
	The function can be splitted into two steps:
	#1 The function is used to find the row which containing the fewest 0.
	#2 Select the zero number on the row, and then marked the element corresponding row and column as False
	'''

	#Find the row
	min_row = [99999, -1]

	for row_num in range(zero_mat.shape[0]):
		if np.sum(zero_mat[row_num] == True) > 0 and min_row[0] > np.sum(zero_mat[row_num] == True):
			min_row = [np.sum(zero_mat[row_num] == True), row_num]

	# Marked the specific row and column as False
	zero_index = np.where(zero_mat[min_row[1]] == True)[0][0]
	mark_zero.append((min_row[1], zero_index))
	zero_mat[min_row[1], :] = False
	zero_mat[:, zero_index] = False

def mark_matrix(mat):

	'''
	Finding the returning possible solutions for LAP problem.
	'''

	#Transform the matrix to boolean matrix(0 = True, others = False)
	cur_mat = mat
	zero_bool_mat = (cur_mat == 0)
	zero_bool_mat_copy = zero_bool_mat.copy()

	#Recording possible answer positions by marked_zero
	marked_zero = []
	while (True in zero_bool_mat_copy):
		min_zero_row(zero_bool_mat_copy, marked_zero)

	#Recording the row and column positions seperately.
	marked_zero_row = []
	marked_zero_col = []
	for i in range(len(marked_zero)):
		marked_zero_row.append(marked_zero[i][0])
		marked_zero_col.append(marked_zero[i][1])

	#Step 2-2-1
	non_marked_row = list(set(range(cur_mat.shape[0])) - set(marked_zero_row))

	marked_cols = []
	check_switch = True
	while check_switch:
		check_switch = False
		for i in range(len(non_marked_row)):
			row_array = zero_bool_mat[non_marked_row[i], :]
			for j in range(row_array.shape[0]):
				#Step 2-2-2
				if row_array[j] == True and j not in marked_cols:
					#Step 2-2-3
					marked_cols.append(j)
					check_switch = True

		for row_num, col_num in marked_zero:
			#Step 2-2-4
			if row_num not in non_marked_row and col_num in marked_cols:
				#Step 2-2-5
				non_marked_row.append(row_num)
				check_switch = True
	#Step 2-2-6
	marked_rows = list(set(range(mat.shape[0])) - set(non_marked_row))

	return(marked_zero, marked_rows, marked_cols)

def adjust_matrix(mat, cover_rows, cover_cols):
	cur_mat = mat
	non_zero_element = []

	#Step 4-1
	for row in range(len(cur_mat)):
		if row not in cover_rows:
			for i in range(len(cur_mat[row])):
				if i not in cover_cols:
					non_zero_element.append(cur_mat[row][i])
	min_num = min(non_zero_element)

	#Step 4-2
	for row in range(len(cur_mat)):
		if row not in cover_rows:
			for i in range(len(cur_mat[row])):
				if i not in cover_cols:
					cur_mat[row, i] = cur_mat[row, i] - min_num
	#Step 4-3
	for row in range(len(cover_rows)):
		for col in range(len(cover_cols)):
			cur_mat[cover_rows[row], cover_cols[col]] = cur_mat[cover_rows[row], cover_cols[col]] + min_num
	return cur_mat

def hungarian_algorithm(mat):
	dim = mat.shape[0]
	cur_mat = mat

	# every column and every row subtract its internal minimum
	for row_num in range(mat.shape[0]):
		cur_mat[row_num] = cur_mat[row_num] - np.min(cur_mat[row_num])

	for col_num in range(mat.shape[1]):
		cur_mat[:,col_num] = cur_mat[:,col_num] - np.min(cur_mat[:,col_num])
	zero_count = 0
	while zero_count < dim:
		#Step 2 & 3
		ans_pos, marked_rows, marked_cols = mark_matrix(cur_mat)
		zero_count = len(marked_rows) + len(marked_cols)

		if zero_count < dim:
			cur_mat = adjust_matrix(cur_mat, marked_rows, marked_cols)

	return ans_pos

def ans_calculation(mat, pos):
	total = 0
	ans_mat = np.zeros((mat.shape[0], mat.shape[1]))
	for i in range(len(pos)):
		total += mat[pos[i][0], pos[i][1]]
		ans_mat[pos[i][0], pos[i][1]] = mat[pos[i][0], pos[i][1]]
	return total, ans_mat

#take in output matrix from hungarian algorithm, the hospital names,and the number of doctors and output a list with a hospital assignment for each doctor
def assignment_list(matrix,col_names,num_doctors):
	assignments = []
	matrix = pd.DataFrame(matrix, columns = col_names)
	for i in range(num_doctors):
		assignments.append((matrix.iloc[i]>0)[matrix.iloc[i]>0].index[0].split('.')[0])
	return assignments

def main():

	'''Hungarian Algorithm:
	Finding the minimum value in linear assignment problem.
	Therefore, we can find the minimum value set in net matrix
	by using Hungarian Algorithm. In other words, the maximum value
	and elements set in cost matrix are available.'''

	ans_pos = hungarian_algorithm(cost_matrix.copy())#Get the element position.
	ans, ans_mat = ans_calculation(cost_matrix, ans_pos)#Get the minimum or maximum value and corresponding matrix.
	assignments = assignment_list(ans_mat, col_names, num_residents)

	#Show the result
	for resident, assignment in zip(residents, assignments):
		print(f"{resident} is assigned to {assignment}")